In [7]:
from sklearn.datasets import fetch_20newsgroups

import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
categories = [
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
#读取数据
df = fetch_20newsgroups(subset='all',categories=categories, shuffle=False, remove=('headers', 'footers', 'quotes'))

In [9]:
labels = df.target
true_k = len(np.unique(labels)) ## This should be 3 in this example
print(true_k)

3


In [10]:
print(df.data[1])

In regards to fractal commpression, I have seen 2 fractal compressed "movies".
They were both fairly impressive.  The first one was a 64 gray scale "movie" of
Casablanca, it was 1.3MB and had 11 minutes of 13 fps video.  It was a little
grainy but not bad at all.  The second one I saw was only 3 minutes but it
had 8 bit color with 10fps and measured in at 1.2MB.

I consider the fractal movies a practical thing to explore.  But unlike many 
other formats out there, you do end up losing resolution.  I don't know what
kind of software/hardware was used for creating the "movies" I saw but the guy
that showed them to me said it took 5-15 minutes per frame to generate.  But as
I said above playback was 10 or more frames per second.  And how else could you
put 11 minutes on one floppy disk?


In [11]:
'''
这是开始提取特征，这里的特征是词频统计。
'''
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.data)
'''
这是开始提取特征，这里的特征是TFIDF特征。
'''
from sklearn.feature_extraction.text import TfidfTransformer # type: ignore
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [12]:
#print(X_vec)

NameError: name 'X_vec' is not defined

In [ ]:
from sklearn.linear_model import LogisticRegression  # 逻辑回归
clf = LogisticRegression().fit(X_vec, df.target)
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = X_vec.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

In [ ]:
'''
使用逻辑回归分类,并做出简单的预测
'''
from sklearn.linear_model import LogisticRegression  # 逻辑回归
clf = LogisticRegression().fit(X_train_tfidf, df.target)
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, df.target_names[category]))

'God is love' => talk.religion.misc
'OpenGL on the GPU is fast' => comp.graphics


In [ ]:
doc = ['i love computer', 'my father is a good man']
X_new_counts = count_vect.transform(doc)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(doc, predicted):
    print('%r => %s' % (doc, df.target_names[category]))

'i love computer' => comp.graphics
'my father is a good man' => talk.religion.misc


In [ ]:
from wordcloud import WordCloud